<a href="https://colab.research.google.com/github/mana-01/G-s-acadmy/blob/master/scraping_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
pip install gspread

In [12]:
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapi.com/auth/drive']
service_account_file = 'gss_api.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(service_account_file, scope)

gs = gspread.authorize(credentials)

spreadsheet_key = '17Q9_9j_1w4F7WFVLKymUTf_IQgMLKDdI9Y0rdKi2F3o'
worksheet = gs.open_by_key(spreadsheet_key).worksheet(スクレイピング)




auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [13]:
from bs4 import BeautifulSoup
import requests
import lxml
import html
from urllib.request import urlopen
import re
import numpy as np
import pandas as pd

workbook_url = "https://docs.google.com/spreadsheets/d/17Q9_9j_1w4F7WFVLKymUTf_IQgMLKDdI9Y0rdKi2F3o/edit#gid=0"
workbook = gc.open_by_url(workbook_url)

with urlopen ("https://store.enterthee.jp/collections/ladies") as res:
  html = res.read().decode("utf-8")

soup = BeautifulSoup(html, "html.parser")
items = soup.select("div.productitem")

print(items[10])

<div class="productitem" data-product-item-content="">
<a class="productitem--image-link" data-product-page-link="" href="/collections/ladies/products/caroline-eco-recycled-black" tabindex="-1">
<figure class="productitem--image" data-product-item-image="">
<noscript data-rimg-noscript="">
<img alt="" class="productitem--image-alternate" data-rimg="noscript" src="//cdn.shopify.com/s/files/1/0271/5802/0178/products/K210701202_6131_2_600x_2e1de141-6e53-46e7-96cc-0973f1fa9368_512x768.jpg?v=1639970416" srcset="//cdn.shopify.com/s/files/1/0271/5802/0178/products/K210701202_6131_2_600x_2e1de141-6e53-46e7-96cc-0973f1fa9368_512x768.jpg?v=1639970416 1x, //cdn.shopify.com/s/files/1/0271/5802/0178/products/K210701202_6131_2_600x_2e1de141-6e53-46e7-96cc-0973f1fa9368_599x899.jpg?v=1639970416 1.17x"/>
</noscript>
<img alt="" class="productitem--image-alternate" data-rimg="lazy" data-rimg-crop="false" data-rimg-max="600x900" data-rimg-scale="1" data-rimg-template="//cdn.shopify.com/s/files/1/0271/580

In [14]:
# 商品名・値段・URLのデータ取得のための絞り込み・確認
iteme_title_list = soup.find_all("h2", attrs= {"class":"productitem--title"})
item_price_list = soup.find_all("span", attrs = {"class":"money price__current--min", "data-price-min":""})
item_url_list = soup.select("h2 a")

In [15]:
product_list = [iteme_title_list.getText().replace("\n", "" )
.replace("1月11日まで受注 1月下旬に発送予定", "")
.replace("受注再開！1月11日まで受注 1月下旬に発送予定","")
.replace("受注再開！","")
.replace("受注再開","")
.replace("30001月11日まで受注受付中1月下旬発送予定","")
.replace("1月11日まで受注受付中1月下旬発送予定","")
.replace("\u3000","")
.strip() for iteme_title_list in soup.find_all({"h2"}, {"class":"productitem--title"})]

        


In [16]:

# 値段のリスト化(最安価格で取得している）
price_list = []
for price in item_price_list:
  price_list.append(price.text)
print(price_list)


['¥15,600', '¥21,300', '¥14,300', '¥13,600', '¥42,800', '¥24,300', '¥24,200', '¥24,200', '¥20,700', '¥20,700', '¥18,400', '¥21,800', '¥18,400', '¥21,300', '¥21,300', '¥20,900', '¥17,900', '¥17,000', '¥45,000', '¥42,800', '¥19,900', '¥7,200', '¥12,900', '¥12,900', '¥17,200', '¥6,400', '¥6,400', '¥6,400', '¥18,400', '¥9,900', '¥12,900', '¥18,400', '¥18,400', '¥18,400', '¥18,400', '¥24,200']


In [17]:
# 詳細URLのリスト化
url_list = []

for url in item_url_list:
  link = url.get("href")
  url_list.append(link)
print(url_list)

['/collections/ladies/products/tt3011-wrap-cardigan', '/collections/ladies/products/coat-moe', '/collections/ladies/products/tt83-shirt-dress-eco-vero', '/collections/ladies/products/forget-blouse-black-fairy-green-art', '/collections/ladies/products/coasts-and-jackets-north-coat-2', '/collections/ladies/products/decima-ecru', '/collections/ladies/products/dimension-ops-knitted-sweater', '/collections/ladies/products/yellow-lada-sweater', '/collections/ladies/products/atis-forget-dress', '/collections/ladies/products/forget-placket-dress', '/collections/ladies/products/caroline-eco-recycled-black', '/collections/ladies/products/alice-eco-recycled-dark-used', '/collections/ladies/products/cardigan-mevik', '/collections/ladies/products/dress-lasila-newmoon', '/collections/ladies/products/dress-lasila', '/collections/ladies/products/asti-placket-dress-1', '/collections/ladies/products/asti-dress-black-bloomy-art', '/collections/ladies/products/stormy-jeans-0108', '/collections/ladies/prod

In [18]:
# 取得した各リストをデータフレームとしてまとめる（テーブル化）
df = pd.DataFrame({"商品名":product_list,
                    "値段":price_list,
                    "詳細URL":url_list})
df

,商品名,値段,詳細URL
0,1月18日締切 1月末‐2月上旬発送予定 オーガニックコットン TT3011ラップカーディガン,"¥15,600",/collections/ladies/products/tt3011-wrap-cardigan
1,オーガニックコットンモエコートブラック,"¥21,300",/collections/ladies/products/coat-moe
2,1月18日締切 1月末‐2月上旬発送予定エコヴェロシャツ ワンピースダークオリーブ,"¥14,300",/collections/ladies/products/tt83-shirt-dress-...
3,1月18日締切 1月末‐2月上旬発送予定リサイクルポリエステルForget ブラウスBlac...,"¥13,600",/collections/ladies/products/forget-blouse-bla...
4,リサイクルポリエステルノースコートグレー,"¥42,800",/collections/ladies/products/coasts-and-jacket...
5,1月18日締切 1月末‐2月上旬発送予定 オーガニックコットン DECIMAニット,"¥24,300",/collections/ladies/products/decima-ecru
6,1月18日締切 1月末‐2月上旬発送予定 エクストラファインウールセーターDIMENSION...,"¥24,200",/collections/ladies/products/dimension-ops-kni...
7,1月18日締切 1月末‐2月上旬発送予定 エクストラファインウールセーターYELLOW L...,"¥24,200",/collections/ladies/products/yellow-lada-sweater
8,1月18日締切 1月末‐2月上旬発送予定 リサイクルポリエステルATIS Forget ワン...,"¥20,700",/collections/ladies/products/atis-forget-dress
9,1月18日締切 1月末‐2月上旬発送予定リサイクルポリエステルATIS Forget pla...,"¥20,700",/collections/ladies/products/forget-placket-dress


In [19]:
# データ形状の確認
df.shape




(36, 3)

In [20]:
# ワークシートへ更新情報の追加
def add_worksheet(title, rows, cols):
  worksheet = workbook.add_worksheet(title=title_text, rows=100, cols=4)
  worksheet.append_row(df)
